#### Example 1 : Simple RAG and its workflow

In [10]:
import langchain
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader,WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain import hub
import bs4
load_dotenv()

True

In [2]:
# loading the env variables require
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

##### 1. Loading of the data

In [3]:
# TextLoader : creating loader object and loading the text
text_loader = TextLoader(file_path="generative_ai.txt")
text = text_loader.load()

In [4]:
# web path
web_path = "https://www.analyticsvidhya.com/blog/2024/09/email-marketing-with-genai-agents/"

# WebBasedLoader : loading the web page texts
web_loader = WebBaseLoader(
    web_paths=[web_path],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_=("common-style-py detail-page pt-4","text-light-primary fs-32 fw-semibold","content-box"))}
    )
docs = []
for doc in web_loader.lazy_load():
    docs.append(doc)


In [7]:
# print(docs[0].page_content)

##### 2. Transforming data using text splitters

RecursiveCharacterTextSplitter --> this splits the text by considering the heirarchical form of document, paragraph,lines,words. try to split the document such a way that the paragraph should not loose its meaning. the lines should be completely splited at the full stop not in between.

In [8]:
# RecursiveCharacterTexrSplitter --> chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
texts = text_splitter.split_documents(docs)

##### 3. Generating the embeddings and store it in the vectorstore

In [9]:
# embeddings model loading

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\admin\anaconda3\envs\env_langchain_tutorial\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# storing the embeddings related to document into the vector database

# creates vectorstore
chroma_db = Chroma.from_documents(documents=texts,embedding=embedding) 

# query the database to get relevent documents from the chroma_db
query = "How agentic email system is different from the traditional email system ?"
results = chroma_db.similarity_search(query)

In [9]:
results

[Document(metadata={'source': 'https://www.analyticsvidhya.com/blog/2024/09/email-marketing-with-genai-agents/'}, page_content='Traditional Email System\nLet’s understand the traditional email system first. Imagine you have a vast internet presence, which includes the organization’s website with multiple web pages, blogs, free downloadables, etc. And you are constantly pursuing expanding your online presence with webinars, online conferences, and access to free e-books and product trials. All of this results in a vast collection of email IDs. A generic traditional marketing system involves:'),
 Document(metadata={'source': 'https://www.analyticsvidhya.com/blog/2024/09/email-marketing-with-genai-agents/'}, page_content='Note: Even at this stage, you cannot call the system autonomous, as it still requires constant human intervention to ensure that the agents improve based on the performance of the emails sent.'),
 Document(metadata={'source': 'https://www.analyticsvidhya.com/blog/2024/09

In [11]:
extracted_text = ""
for res in results:
    extracted_text+=res.page_content

##### 4. Pass the query + extracted_text to the LLM model

In [15]:
# load the meta llama3 model with the help of groq inference engine
model_id = "llama-3.3-70b-versatile"
prompt_template = ChatPromptTemplate(
    messages=[
        ('system','You are a helpful assistant, who answers the questions related to Generative AI and its uses'),
        ('user','{query}')
    ]
    )
prompt = prompt_template.invoke(query)
model = ChatGroq(
    model=model_id,
    temperature=0.5,
    max_retries=2,
)

In [24]:
AI_response = model.invoke(prompt).content

In [26]:
print(AI_response)

An agentic email system is a type of email system that utilizes artificial intelligence (AI) and machine learning (ML) to manage and process emails in a more autonomous and proactive way. Here are some key differences between an agentic email system and a traditional email system:

1. **Autonomy**: Agentic email systems can perform tasks independently, such as sorting, prioritizing, and responding to emails, without the need for human intervention. Traditional email systems, on the other hand, rely on manual management by the user.
2. **Intelligence**: Agentic email systems use AI and ML algorithms to analyze emails, identify patterns, and make decisions. Traditional email systems lack this level of intelligence and rely on simple rules-based filtering.
3. **Proactivity**: Agentic email systems can anticipate and take action on emails before the user even sees them. For example, they can automatically schedule meetings, send reminders, or respond to routine inquiries. Traditional email